In [2]:
import pandas as pd
import numpy as np

import traceback
import psycopg2

from datetime import datetime
from datetime import timedelta
import os
import re 

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [2]:
date = datetime.now().strftime('%d-%m-%Y')

## Patterns

In [3]:
national_life_pattern = re.compile(r'NLG_AllNewBusinessReport.*.xlsx')

In [4]:
listdir = os.listdir('//mnt/g/Shared drives/Operations Dept/02. PROCESSES/01. BOB/02. LIFE/2023/Backup National Life')

FileNotFoundError: [Errno 2] No such file or directory: '//mnt/g/Shared drives/Operations Dept/02. PROCESSES/01. BOB/02. LIFE/2023/Backup National Life'

# National Life

## Functions

Función para el separado de los nombres en las columnas Menber First Name y Merber Last Name. Separa los nombres de los apellidos

In [7]:
def split_names(name:str, first:bool=True, split_by=' ') -> str:
    name = name.split(split_by, maxsplit=1)
    if first:
        return name[0].replace(',','')
    return name[1].replace(',','')

Homologa los status del carrier a los que se usan en el BOB

In [8]:
def replace_status(x):
    try:
        if 'Pending EFT -' in x:
            x = x.split('-')[0].strip()
            
        replaced_status = df_status[df_status['Status Carrier'] == x]['Status SF'].values[0]
        return replaced_status
    except IndexError:
        return None

### Conection

In [9]:
def start():
    connection = psycopg2.connect(dbname = 'Carriers',
                    host = '188.166.63.213',
                    port = '5432',
                    user = 'cperez',
                    password = 'srb4o7SlqAIf')

    return connection

Función que se usa para hacer queries a la base de datos

In [10]:
def query_db (query):
    try:
        connection = start()
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
        connection.commit()
        return result
    except Exception as ex:
        traceback.print_exc(ex)
    finally:
        connection.close()

        

In [11]:
connection = start()

### Carga

In [163]:
df_b = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/bob_20230110.csv', encoding='latin-1', low_memory=False)
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
0,BOB# 00000001,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192951620161,Direct,...,NaN,Silver,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyj,
1,BOB# 00000002,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921397071,Direct,...,NaN,Bronze,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyk,
2,BOB# 00000003,Miguel Bautista,Molina,1647487.0,AMARILYS PEREZ LAZO,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921385481,Direct,...,NaN,Bronze,NaN,12/20/2022,V-Actualización 09-01,Renewals,NaN,Molina Florida,a1C1G000007eLyl,


In [97]:
df_a = (pd.read_excel('//mnt/c/Users/cesar/Desktop/claro/life/NationalLife/NLG_AllNewBusinessReport_01122023.xlsx', header=5)
                     .dropna(axis='columns', how='all')
                     .dropna( how='all'))

df_a = df_a.iloc[:-3,:]

In [98]:
df_a.head()

,Policy #,Insured / Annuitant,Agent,Status,Delivery,Action,Submitted,Modal Premium,Product,Sent,Owner,Submit Method,Case Manager,AAP*,Agent #,Agency,Company Code
0,LS759648800,"Garcia, Coralia Garmendia",GUERRERO VASQUEZ LUZ EUGENIA,PENDING,-,NaN,01/11/2023,88.44,LSW Level Term 20-G,NaN,Coralia Garmendia Garcia,e-APP,Danzavian Lamb,1005,3921T,INFINITY MARKETING - 517,LS
1,LS759567900,"Ramirez, Juan Olivares",RAMIREZ SUQUINO DUNIA,PENDING,-,[01/07/2023] NEWAGT,01/07/2023,76.56,LSW Level Term 30-G,NaN,Juan Olivares Ramirez,e-APP,Tyler Everett,870,5039U,INFINITY MARKETING - 517,LS
2,LS759326300,"Livelli, Julia Carlini",GRANADILLO LORENA,PENDING,-,[01/09/2023] EMAIL FROM EMILY,01/04/2023,35.02,LSW Level Term 15-G,NaN,Julia Carlini Livelli,e-APP,Holly Clark,398,5168S,INFINITY MARKETING - 517,LS
3,LS140058700,"Ferras, Antonella Calcano",GRANADILLO LORENA,PENDING,-,[12/30/2022] APS ORDERED HO\n[12/29/2022] EMAI...,12/23/2022,500,FlexLife,NaN,Roberto Calcano,e-APP,Holly Clark,2975.94,5168S,INFINITY MARKETING - 517,LS
4,LS139893600,"Valdez, Lilian",REYNOSO MIGUEL ANGEL,PENDING,-,[12/27/2022] PARAMED EXAM REQ'D\n[12/27/2022] ...,12/21/2022,186,FlexLife,NaN,Lilian Valdez,e-APP,Tyler Everett,2232,2993U,INFINITY MARKETING - 517,LS


## Homologación

In [99]:
df_a.columns

Index(['Policy #', 'Insured / Annuitant', 'Agent', 'Status', 'Delivery',
       'Action', 'Submitted', 'Modal Premium', 'Product', 'Sent', 'Owner',
       'Submit Method', 'Case Manager', 'AAP*', 'Agent #', 'Agency',
       'Company Code'],
      dtype='object')

In [100]:
df_a = df_a.rename(
    columns={
        'Policy #':'PolicyNumber',
        'Insured / Annuitant':'Insured',
        'Agent':'Agent AOR',
        'Status':'Status',
        'Submitted':'EffectiveDate',
        'AAP*': 'FinalPremium',
        'Agent #':'AgentNumber'
        
        
    }
)

### Limpiar Policy Number

Queitar los prefijos a los policy numbers del carriers

In [101]:
def limpiarPolicyNumber(df, column_name, replace_string:str='LS'):
    df = df[column_name].str.replace(replace_string,'')
    return df

In [102]:
df_a['PolicyNumberClean'] = limpiarPolicyNumber(df_a, 'PolicyNumber')

### Extraer las polizas de National Life

Extraer carrier

In [103]:
def extractCarrier(df, carrier_name, carrier_column):
    df =  df[df[carrier_column] == carrier_name]
    return df

In [104]:
df_b = extractCarrier(df_b, 'National Life', 'Carrier')

### Hacer join

In [105]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [106]:
upload_a = leftJoinBOB(df_a['PolicyNumberClean'].values, df_b['Policy Number'].values)

In [107]:
upload_b = leftJoinBOB(df_a['PolicyNumber'].values, df_b['Policy Number'].values)

In [108]:
upload = set(upload_a + [i.replace('LS', '') for i in upload_b])

In [109]:
df_upload = df_a[df_a['PolicyNumberClean'].isin(upload)]

### Upload into template

In [110]:
df_upload.reset_index(drop=True, inplace=True)

In [3]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)

In [5]:
df_template.columns

Index(['Carrier___', 'LOB____', 'Carrier', 'Agent AOR', 'NPN AOR',
       'Contact____', 'NPN_Internal____', 'Contact', 'Policy Number', 'Source',
       'Referrals_____', 'Referrals Id', 'Status', 'Effective Date',
       'Transaction Date', 'Policy Term Date', 'Member First Name',
       'Member Last Name', 'DOB', 'State', 'County', 'City', 'Members',
       'Policy Type', 'Last Status Modification Date', 'Loading Date',
       'Record Type Id', 'Plan', 'Election Period', 'Final Premium', 'Subsidy',
       'Annual Premium', 'Paid Thru Date', 'Days Past Due', 'Phone Number',
       'Email', 'Metal Level', 'Zip Code', 'Claro Type',
       'Carrier State line of Business'],
      dtype='object')

In [113]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))

In [114]:
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [115]:
df_template.reset_index(drop=True, inplace=True)

In [116]:
df_template['Policy Number'] = df_upload['PolicyNumberClean']

In [117]:
df_template['Final Premium'] = df_upload['FinalPremium']

In [118]:
df_template['Plan'] = df_upload['Product']

In [119]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [120]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [121]:
df_template['LOB____'] = 'Life'

#### Source

Dependiendo si el npn se encuentra en la lista de npn´s referenciales, el source va a ser directo (En caso de que no se encuentre), o referencial

In [122]:
def assign_source(referal_agents:list, df_template, column_agent_numbers:str, df_upload) -> pd.DataFrame:
    df_s = df_template.copy()
    mask_s = df_upload[column_agent_numbers].isin(referal_agents)
    
    df_s.loc[mask_s,  'Source'] = 'Referal'
    df_s.loc[~mask_s, 'Source'] = 'Direct'
    
    return df_s

In [123]:
def getSource(query,connection, df_template, df_upload, column_agent_numbers='AgentNumber') -> pd.DataFrame:
    referal_agents = [i[0] for i in query_db(query)]
    referal_list_agents = list(set(referal_agents).intersection(set(df_upload[column_agent_numbers].values)))
    
    df = assign_source(referal_list_agents, df_template, column_agent_numbers, df_upload)
    return df

In [124]:
npn_referals = [18233171, 7388594, 19603471, 18522178, 19106367, 16001347, 10041154, 20462802, 17686050]

npn_referals = str([str(i)  for i in npn_referals]).replace('[', '').replace(']','')


In [125]:
query_referals_agents = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn in ({list_refarals}) ;
""".format(list_refarals=npn_referals)

In [126]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

Siempre es uno

In [127]:
df_template['Members'] = 1

#### State

Siempre es Florida

In [128]:
df_template['State'] = 'FL'

#### Record Type

Si la poliza es de Jose m Parra, el record types es Direct Sales, de lo contrario es BOB

In [129]:
def recordType(df_upload, df_template, column_name='Record Type Id', column_agent='AgentNumber'):
    
    record_types = dict(query_db(query_record_type))
    agent_writing_number = query_db(query_awn_jose_m)
    agent_writing_number_jose_m = [i[0] for i in agent_writing_number]
    
#     jose_m_parra_awn_in_df = list(set(agent_writing_number).intersection(set(df_upload[column_agent].values)))
    others_awn = list(set(df_upload[column_agent].values) - set(agent_writing_number_jose_m))
    
    df_r = df_template.copy()
    mask = df_upload[column_agent].isin(others_awn)
    df_r.loc[mask, column_name] = record_types['BOB']
    df_r.loc[~mask, column_name] = record_types['Direct Sales']
    
    return df_r

In [130]:
query_record_type = """
    SELECT record_type, id  FROM record_type rt ;
"""

In [131]:
query_awn_jose_m = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn = '17795437'
    ;
"""

In [132]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [133]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

Se homologa la columnas de carrier a las del BOB

In [134]:
df_status = pd.read_excel('/mnt/c/Users/cesar/Desktop/claro/life/Lista Status Life.xlsx')
df_status.head(3)

,Status Carrier,Status SF
0,Active,Active
1,APPROVED,Approved
2,Approved/Admin Review,Approved


In [135]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

Si la poliza tiene más de 365 días y es renewal

In [136]:
def compare_years_policy(x): 
    one_year_ago = datetime.now() - timedelta(days=365)
    if one_year_ago >= x:
        return 'Renewal'
    return 'New Sale'

In [137]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [138]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [139]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [140]:
df_template['Loading Date'] = date

#### NPN AOR

De base de datos se extrae el npn del agente

In [141]:
query_agent_AOR = """
    SELECT contact, npn, agent_writing_number, id
    FROM agent_contract
    ;
"""

In [142]:
def extract_agent_aor(x, df_contact, column_name='NPN'):
    result = df_contact[df_contact['AgentWritingNumber'] == x][column_name].unique()
    assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
    if len(result) > 0:
        return result[0]
    return None

In [143]:
def npnAOR(query_agent_AOR,connection, agent_number_colunm_name='AgentNumber'):
    list_contact = query_db(query_agent_AOR)
    df_contact = pd.DataFrame([{'Contact Name':i[0], 'NPN':i[1], 'AgentWritingNumber':i[2], 'Contact ID':i[3]} for i in list_contact])
    
    
    df_template['NPN AOR'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact))
    df_template['Contact'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact, 'Contact ID'))
    return df_template, df_contact

In [144]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

#### Contact ID

In [145]:
# def extract_contact_id(x, df_contact):
#     result = df_contact[df_contact['AgentWritingNumber'] == x]['Contact ID'].unique()
#     assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
#     if len(result) > 0:
#         return result[0]
#     return None

In [146]:
# df_template['Contact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Carrier State Line of Business

De base de datos de extrae el id del carrier state line of business

In [147]:
query_cslb = """
    select carrier_lob_id 
    from carrier c
    where carrier_id = '{carrier_id}' and state_c = '{state}';
"""

In [148]:
def making_query_cslb(df_template, query_cslb):
    carrier = df_template['Carrier'].unique()
    state = df_template.State.unique()
    
    assert (len(carrier) == 1) and (len(state) == 1), 'Error en la integridad de los datos.'
    
    query_cslb = query_cslb.format(carrier_id=carrier[0], state=state[0])
    value = query_db(query_cslb)[0][0]
    return value

In [149]:
def extract_cslb(df_template, query_cslb):
    cslb = making_query_cslb(df_template, query_cslb)
    df_template['Carrier State line of Business'] = cslb
    
    return df_template

In [150]:
df_template = extract_cslb(df_template, query_cslb)

## Time

In [151]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date',  'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [152]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

In [157]:
df_faltantes = df_template[df_template['Contact'].isna()]

In [154]:
def iterpolate(df, name, new_fields):
    df_t = df.copy()
    df_t.loc[df['Agent AOR'] == name, ['NPN AOR', 'Contact']] = new_fields
    return df_t

In [158]:
df_faltantes

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name


## Interpolacion

In [156]:
df_template = iterpolate(df_template, 'RAMIREZ SUQUINO DUNIA',  ('20544688','0031G00001YIab5QAD'))

In [93]:
#df_template = iterpolate(df_template, 'RIVERA JAIDIRA D',  ('18915809','0031G00000vqvIwQAI'))

In [94]:
#df_template = iterpolate(df_template, 'OROZCO ANGELA PATRICIA',  ('9709106','0031G00000vq9aHQAQ'))

In [95]:
#df_second_upload = df_template[df_template['Agent AOR'].isin(['COUTO ROSANGELA ROCHA', 'RIVERA JAIDIRA D', 'OROZCO ANGELA PATRICIA'])]

## Load

In [160]:
df_template.shape

(9, 20)

In [159]:
df_template.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_national_life{date}_.csv', index=False)

In [987]:
df_second_upload.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_national_life{date}_faltantes_agent_aor.csv', index=False)

# Ameritas

In [161]:
def limpiar_df(df) -> pd.DataFrame:
    df = df.dropna(axis='columns', how='all').dropna( how='all')
    return df

In [164]:
df_ameritas = limpiar_df(pd.read_excel('//mnt/c/Users/cesar/Desktop/claro/life/Ameritas/Ameritas 01-16-2023.xls'))
df_ameritas.head()

,Insured Name,Company Code,Policy Number,Writing Agent Name,Writing Agent ID,Writing Agent Profile,Servicing Agent Name,Servicing Agent ID,Servicing Agent Profile,Product Name,Issue Date,Policy Status
0,"AMADOR, JULLIANNA",100,CI60002150,"ARAQUE, RONNY",AG00105858,1,"ARAQUE, RONNY",AG00105858,1,FLX Living Benefits Index UL Guideline Premium...,04/15/2022,Active/Premium Paying
1,"DIAZ, EDWARD A",100,CI10030745,"RAMOS, WALTER",AG00122703,1,"RAMOS, WALTER",AG00122703,1,FLX Living Benefits Index UL Guideline Premium...,05/18/2021,Active/Premium Paying
2,"GONZALEZ, DANIELA F",100,CI10032287,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Surrendered
3,"MURILLO, RAFAEL A",100,CI10032145,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Active/Premium Paying
4,"MURILLO, VIRNA A",100,CI10033924,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,10/01/2021,Active/Premium Paying


## Homologación

In [165]:
df_ameritas.columns

Index(['Insured Name', 'Company Code', 'Policy Number', 'Writing Agent Name',
       'Writing Agent ID', 'Writing Agent Profile', 'Servicing Agent Name',
       'Servicing Agent ID', 'Servicing Agent Profile', 'Product Name',
       'Issue Date', 'Policy Status'],
      dtype='object')

In [166]:
df_ameritas = df_ameritas.rename(
    columns={
        'Policy Number':'PolicyNumber',
        'Insured Name':'Insured',
        'Writing Agent Name':'Agent AOR',
        'Policy Status':'Status',
        'Issue Date':'EffectiveDate',
        'Writing Agent ID':'AgentNumber',
        'Product Name':'Product'
        
        
    }
)

### Limpiar Policy Number

In [167]:
# df_a['PolicyNumberClean'] = limpiarPolicyNumber(df_a, 'PolicyNumber')

### Extraer las polizas de National Life

In [168]:
df_b = extractCarrier(df_b, 'Ameritas', 'Carrier')

### Hacer join

In [177]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [178]:
upload = leftJoinBOB(df_ameritas['PolicyNumber'].values, df_b['Policy Number'].values)

In [179]:
df_upload = df_ameritas[df_ameritas['PolicyNumber'].isin(upload)]

In [180]:
df_upload

,Insured,Company Code,PolicyNumber,Agent AOR,AgentNumber,Writing Agent Profile,Servicing Agent Name,Servicing Agent ID,Servicing Agent Profile,Product,EffectiveDate,Status


### Upload into template

In [173]:
df_upload.reset_index(drop=True, inplace=True)

In [175]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)
df_template

In [1003]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [1004]:
df_template.reset_index(drop=True, inplace=True)

In [1005]:
df_template['Policy Number'] = df_upload['PolicyNumber']

In [1006]:
# df_template['Final Premium'] = df_upload['FinalPremium']

In [1007]:
df_template['Plan'] = df_upload['Product']

In [1008]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [1009]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [1010]:
df_template['LOB____'] = 'Life'

#### Source

In [1011]:
def assign_source(referal_agents:list, df_template, column_agent_numbers:str, df_upload) -> pd.DataFrame:
    df_s = df_template.copy()
    mask_s = df_upload[column_agent_numbers].isin(referal_agents)
    
    df_s.loc[mask_s,  'Source'] = 'Referal'
    df_s.loc[~mask_s, 'Source'] = 'Direct'
    
    return df_s

In [1012]:
def getSource(query,connection, df_template, df_upload, column_agent_numbers='AgentNumber') -> pd.DataFrame:
    referal_agents = [i[0] for i in query_db(query)]
    referal_list_agents = list(set(referal_agents).intersection(set(df_upload[column_agent_numbers].values)))
    
    df = assign_source(referal_list_agents, df_template, column_agent_numbers, df_upload)
    return df

In [1013]:
npn_referals = [18233171, 7388594, 19603471, 18522178, 19106367, 16001347, 10041154, 20462802, 17686050]

npn_referals = str([str(i)  for i in npn_referals]).replace('[', '').replace(']','')


In [1014]:
query_referals_agents = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn in ({list_refarals}) ;
""".format(list_refarals=npn_referals)

In [1015]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

In [1016]:
df_template['Members'] = 1

#### Record Type ID

In [1017]:
# df_template['Record Type Id']

#### State

In [1018]:
df_template['State'] = 'FL'

#### Record Type

In [1019]:
def recordType(df_upload, df_template, column_name='Record Type Id', column_agent='AgentNumber'):
    
    record_types = dict(query_db(query_record_type))
    agent_writing_number = query_db(query_awn_jose_m)
    agent_writing_number_jose_m = [i[0] for i in agent_writing_number]
    
#     jose_m_parra_awn_in_df = list(set(agent_writing_number).intersection(set(df_upload[column_agent].values)))
    others_awn = list(set(df_upload[column_agent].values) - set(agent_writing_number_jose_m))
    
    df_r = df_template.copy()
    mask = df_upload[column_agent].isin(others_awn)
    df_r.loc[mask, column_name] = record_types['BOB']
    df_r.loc[~mask, column_name] = record_types['Direct Sales']
    
    return df_r

In [1020]:
query_record_type = """
    SELECT record_type, id  FROM record_type rt ;
"""

In [1021]:
query_awn_jose_m = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn = '17795437'
    ;
"""

In [1022]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [1023]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

In [1024]:
df_status = pd.read_excel('/mnt/c/Users/cesar/Desktop/claro/life/Lista Status Life.xlsx')
df_status.head(3)

,Status Carrier,Status SF
0,Active,Active
1,APPROVED,Approved
2,Approved/Admin Review,Approved


In [1025]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

In [1026]:
def compare_years_policy(x): 
    one_year_ago = datetime.now() - timedelta(days=365)
    if one_year_ago >= x:
        return 'Renewal'
    return 'New Sale'

In [1027]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [1028]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [1029]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [1030]:
df_template['Loading Date'] = date

#### NPN AOR

In [1115]:
query_agent_AOR = """
    SELECT contact, npn, agent_writing_number, id
    FROM agent_contract
    ;
"""

In [1116]:
def extract_agent_aor(x, df_contact, column_name='NPN'):
    result = df_contact[df_contact['AgentWritingNumber'] == x][column_name].unique()
    assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
    if len(result) > 0:
        return result[0]
    return None

In [1117]:
def npnAOR(query_agent_AOR,connection, agent_number_colunm_name='AgentNumber'):
    list_contact = query_db(query_agent_AOR)
    df_contact = pd.DataFrame([{'Contact Name':i[0], 'NPN':i[1], 'AgentWritingNumber':i[2], 'Contact ID':i[3]} for i in list_contact])
    
    
    df_template['NPN AOR'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact))
    df_template['Contact'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact, 'Contact ID'))
    return df_template, df_contact

In [1034]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

In [1035]:
df_upload['AgentNumber']

0    AG00132082
1    AG00132082
Name: AgentNumber, dtype: object

#### Contact ID

In [1036]:
def extract_contact_id(x, df_contact):
    result = df_contact[df_contact['AgentWritingNumber'] == x]['Contact ID'].unique()
    assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
    if len(result) > 0:
        return result[0]
    return None

In [1037]:
# df_template['Cotact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Agent Name

In [1038]:
df_template['Agent Name'] = df_upload['Agent AOR']

#### Carrier State Line of Business

In [1039]:
query_cslb = """
    select carrier_lob_id 
    from carrier c
    where carrier_id = '{carrier_id}' and state_c = '{state}';
"""

In [1040]:
def making_query_cslb(df_template, query_cslb):
    carrier = df_template['Carrier'].unique()
    state = df_template.State.unique()
    
    assert (len(carrier) == 1) and (len(state) == 1), 'Error en la integridad de los datos.'
    
    query_cslb = query_cslb.format(carrier_id=carrier[0], state=state[0])
    value = query_db(query_cslb)[0][0]
    return value

In [1041]:
def extract_cslb(df_template, query_cslb):
    cslb = making_query_cslb(df_template, query_cslb)
    df_template['Carrier State line of Business'] = cslb
    
    return df_template

In [1042]:
df_template = extract_cslb(df_template, query_cslb)

In [1043]:
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business,Agent Name
0,NaN,Life,a0S1G000006iXuWUAU,"CASTANO PADILLA, FATIMA",None,NaN,NaN,None,CI60006169,Direct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Sale,a091G00000B5DUcQAN,"CASTANO PADILLA, FATIMA"
1,NaN,Life,a0S1G000006iXuWUAU,"CASTANO PADILLA, FATIMA",None,NaN,NaN,None,CI60006528,Direct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Sale,a091G00000B5DUcQAN,"CASTANO PADILLA, FATIMA"


#### Ajustar faltantes

In [1044]:
df_template = iterpolate(df_template, 'CASTANO PADILLA, FATIMA',  ('18956578','0031G00001JkjjCQAR'))

In [1048]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,"CASTANO PADILLA, FATIMA",18956578,0031G00001JkjjCQAR,a0S1G000006iXuWUAU,FL,CI60006169,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Direct,None,09/16/2022,FLX Living Benefits Index UL Guideline Premium...,1,06-01-2023,06-01-2023,NaN,JULIO,SANTOS
1,"CASTANO PADILLA, FATIMA",18956578,0031G00001JkjjCQAR,a0S1G000006iXuWUAU,FL,CI60006528,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Direct,Terminated,10/15/2022,FLX Living Benefits Index UL Guideline Premium...,1,06-01-2023,06-01-2023,NaN,MEDINA JORGE R,TORRES


## Load

In [1046]:
df_bk = df_template.copy()

In [1047]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date',  'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [1052]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

In [1053]:
df_template.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_ameritas{date}_.csv', index=False)

In [1063]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,"CASTANO PADILLA, FATIMA",18956578,0031G00001JkjjCQAR,a0S1G000006iXuWUAU,FL,CI60006169,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Direct,NaN,09/16/2022,FLX Living Benefits Index UL Guideline Premium...,1,01/06/2023,01/06/2023,NaN,JULIO,SANTOS
1,"CASTANO PADILLA, FATIMA",18956578,0031G00001JkjjCQAR,a0S1G000006iXuWUAU,FL,CI60006528,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Direct,Terminated,10/15/2022,FLX Living Benefits Index UL Guideline Premium...,1,01/06/2023,01/06/2023,NaN,MEDINA JORGE R,TORRES


In [1055]:
df_template['Status']

0          None
1    Terminated
Name: Status, dtype: object

In [1068]:
revisar_df = df_template[df_template['Policy Number'] == 'CI60006169']

In [1247]:
df_template['Status']

0       Pending
1    Terminated
Name: Status, dtype: object

In [1248]:
revisar_df['Status'] = 'Active'

<ipython-input-1248-dd1483f5ab0b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revisar_df['Status'] = 'Active'


In [1250]:
revisar_df.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/revisar_{date}_.csv', index=False)

# America National

In [182]:
df_b = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/bob_20230110.csv', encoding='latin-1', low_memory=False)
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
0,BOB# 00000001,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192951620161,Direct,...,NaN,Silver,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyj,
1,BOB# 00000002,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921397071,Direct,...,NaN,Bronze,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyk,
2,BOB# 00000003,Miguel Bautista,Molina,1647487.0,AMARILYS PEREZ LAZO,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921385481,Direct,...,NaN,Bronze,NaN,12/20/2022,V-Actualización 09-01,Renewals,NaN,Molina Florida,a1C1G000007eLyl,


In [252]:
df_a = (pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/life/AmricaNational/OR_New America National 01-12-2023.csv')
                     .dropna(axis='columns', how='all')
                     .dropna( how='all'))

In [254]:
df_a.head()

,Agent Name,Policy Type,Policy Number,Critical,Last Update,Product,Annual Premium,Face Amount,State,Applicant,Status,NIA,Requirement,Mailed,Allotment
0,LUISA ALVAREZ,Life New Business,E0238164,No,12/27/2022,Signature Term - 30 Yr,"$1,902.48","$500,000.00",FL,Vicky Cepeda,Inforce,Yes,No,Yes,No
1,ANDRES FELIPE GONZALEZ,Life New Business,E0247216,No,01/04/2023,Signature Term - 20 Yr,$508.08,"$500,000.00",FL,Sonia Ramirez,Waiting for Requirements,No,Yes,Yes,No
2,ANDRES FELIPE GONZALEZ,Life New Business,E0247201,No,01/06/2023,Signature Term - 20 Yr,$648.00,"$500,000.00",FL,Juan Rojas,Waiting for Requirements,No,Yes,Yes,No
3,ANGELA P OROZCO,Life New Business,E0242129,No,12/22/2022,Signature Term - 20 Yr,$269.52,"$500,000.00",FL,Carolina Villegas,Inforce,Yes,No,Yes,No
4,DORILUZ PINA,Life New Business,E0247252,No,01/11/2023,Signature Term - 20 Yr,"$1,024.92","$150,000.00",FL,Santos Mendoza,Waiting for Requirements,No,Yes,Yes,No


## Homologación

In [255]:
df_a.columns

Index(['Agent Name', 'Policy Type', 'Policy Number', 'Critical', 'Last Update',
       'Product', 'Annual Premium', 'Face Amount', 'State', 'Applicant',
       'Status', 'NIA', 'Requirement', 'Mailed', 'Allotment'],
      dtype='object')

In [256]:
df_a = df_a.rename(
    columns={
        'Policy Number':'PolicyNumber',
        'Applicant':'Insured',
        'Agent Name':'Agent AOR',
        'Status':'Status',
        'Last Update':'EffectiveDate',
        'Annual Premium': 'FinalPremium'        
        
    }
)

### Extraer las polizas de National Life

In [258]:
df_b = extractCarrier(df_b, 'American National', 'Carrier')

In [259]:
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
24550,BOB# 00024753,Luigi Fuentes,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,UE095096,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFH,
24551,BOB# 00024754,,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,E0125027,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFI,
24552,BOB# 00024755,Luigi Fuentes,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,UE095094,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFJ,


### Hacer join

In [260]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [263]:
upload = leftJoinBOB(df_a['PolicyNumber'].values, df_b['Policy Number'].values)

In [264]:
df_upload = df_a[df_a['PolicyNumber'].isin(upload)]

In [265]:
df_upload.shape

(7, 15)

In [267]:
df_upload

,Agent AOR,Policy Type,PolicyNumber,Critical,EffectiveDate,Product,FinalPremium,Face Amount,State,Insured,Status,NIA,Requirement,Mailed,Allotment
0,LUISA ALVAREZ,Life New Business,E0238164,No,12/27/2022,Signature Term - 30 Yr,"$1,902.48","$500,000.00",FL,Vicky Cepeda,Inforce,Yes,No,Yes,No
1,ANDRES FELIPE GONZALEZ,Life New Business,E0247216,No,01/04/2023,Signature Term - 20 Yr,$508.08,"$500,000.00",FL,Sonia Ramirez,Waiting for Requirements,No,Yes,Yes,No
2,ANDRES FELIPE GONZALEZ,Life New Business,E0247201,No,01/06/2023,Signature Term - 20 Yr,$648.00,"$500,000.00",FL,Juan Rojas,Waiting for Requirements,No,Yes,Yes,No
3,ANGELA P OROZCO,Life New Business,E0242129,No,12/22/2022,Signature Term - 20 Yr,$269.52,"$500,000.00",FL,Carolina Villegas,Inforce,Yes,No,Yes,No
4,DORILUZ PINA,Life New Business,E0247252,No,01/11/2023,Signature Term - 20 Yr,"$1,024.92","$150,000.00",FL,Santos Mendoza,Waiting for Requirements,No,Yes,Yes,No
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,Life New Business,E0247277,No,01/09/2023,Signature Term - 20 Yr,"$2,553.12","$250,000.00",FL,Gerardo Nunez,Waiting for Requirements,No,Yes,Yes,No
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,Life New Business,E0247238,Yes,12/07/2022,Signature Term - 20 Yr,"$5,804.04","$600,000.00",FL,Gerardo Nunez,Waiting for Requirements,No,Yes,Yes,No


### Upload into template

In [272]:
df_upload.reset_index(drop=True, inplace=True)

In [273]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)

In [274]:
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [275]:
df_upload

,Agent AOR,Policy Type,PolicyNumber,Critical,EffectiveDate,Product,FinalPremium,Face Amount,State,Insured,Status,NIA,Requirement,Mailed,Allotment
0,LUISA ALVAREZ,Life New Business,E0238164,No,12/27/2022,Signature Term - 30 Yr,"$1,902.48","$500,000.00",FL,Vicky Cepeda,Inforce,Yes,No,Yes,No
1,ANDRES FELIPE GONZALEZ,Life New Business,E0247216,No,01/04/2023,Signature Term - 20 Yr,$508.08,"$500,000.00",FL,Sonia Ramirez,Waiting for Requirements,No,Yes,Yes,No
2,ANDRES FELIPE GONZALEZ,Life New Business,E0247201,No,01/06/2023,Signature Term - 20 Yr,$648.00,"$500,000.00",FL,Juan Rojas,Waiting for Requirements,No,Yes,Yes,No
3,ANGELA P OROZCO,Life New Business,E0242129,No,12/22/2022,Signature Term - 20 Yr,$269.52,"$500,000.00",FL,Carolina Villegas,Inforce,Yes,No,Yes,No
4,DORILUZ PINA,Life New Business,E0247252,No,01/11/2023,Signature Term - 20 Yr,"$1,024.92","$150,000.00",FL,Santos Mendoza,Waiting for Requirements,No,Yes,Yes,No
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,Life New Business,E0247277,No,01/09/2023,Signature Term - 20 Yr,"$2,553.12","$250,000.00",FL,Gerardo Nunez,Waiting for Requirements,No,Yes,Yes,No
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,Life New Business,E0247238,Yes,12/07/2022,Signature Term - 20 Yr,"$5,804.04","$600,000.00",FL,Gerardo Nunez,Waiting for Requirements,No,Yes,Yes,No


In [276]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))

In [277]:
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [278]:
df_template.reset_index(drop=True, inplace=True)

In [279]:
df_template['Policy Number'] = df_upload['PolicyNumber']

In [280]:
df_template['Final Premium'] = df_upload['FinalPremium']

In [281]:
df_template['Plan'] = df_upload['Product']

In [282]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [283]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [284]:
df_template['LOB____'] = 'Life'

In [286]:
#df_an = df_upload.copy()

#df_an['AgentNumber'] = 'Q01P6'

In [209]:
df_upload = df_an

#### Source

In [210]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

In [211]:
df_template['Members'] = 1

#### Record Type ID

In [212]:
# df_template['Record Type Id']

#### State

In [213]:
df_template['State'] = 'FL'

#### Record Type

In [214]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [215]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

In [216]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

In [217]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [218]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [219]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [220]:
df_template['Loading Date'] = date

#### NPN AOR

In [244]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

In [245]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,LUISA ALVAREZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0238164,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/27/2022,Signature Term - 30 Yr,1,01/16/2023,01/16/2023,1902.48,Cepeda,Vicky
1,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247216,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/04/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,508.08,Ramirez,Sonia
2,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247201,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/06/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,648.00,Rojas,Juan
3,ANGELA P OROZCO,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0242129,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/22/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,269.52,Villegas,Carolina
4,DORILUZ PINA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247252,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/11/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,1024.92,Mendoza,Santos
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247277,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/09/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,2553.12,Nunez,Gerardo
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247238,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,12/07/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,5804.04,Nunez,Gerardo


#### Contact ID

In [222]:
# df_template['Contact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Agent Name

In [223]:
df_template['Agent Name'] = df_upload['Agent AOR']

In [224]:
df_template.dropna(axis='columns', how='all').to_excel(f'/mnt/c/Users/cesar/Desktop/claro/test_life_{date}_.xlsx', index=False)

#### Carrier State Line of Business

In [225]:
df_template = extract_cslb(df_template, query_cslb)

### Check faltante

In [234]:
df_faltantes = df_template[df_template['Contact'].isna()]

In [235]:
df_faltantes

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name


## Interpolacion

In [236]:
#df_template = iterpolate(df_template, 'RAMIREZ SUQUINO DUNIA',  ('20544688','0031G00001YIab5QAD'))

## Load

In [226]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date', 'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [227]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

In [230]:
df_template['Final Premium'] = df_template['Final Premium'].apply(lambda x : x.replace('$','').replace(',',''))

In [239]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,LUISA ALVAREZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0238164,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/27/2022,Signature Term - 30 Yr,1,01/16/2023,01/16/2023,1902.48,Cepeda,Vicky
1,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247216,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/04/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,508.08,Ramirez,Sonia
2,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247201,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/06/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,648.00,Rojas,Juan
3,ANGELA P OROZCO,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0242129,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/22/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,269.52,Villegas,Carolina
4,DORILUZ PINA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247252,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/11/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,1024.92,Mendoza,Santos
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247277,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/09/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,2553.12,Nunez,Gerardo
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247238,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,12/07/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,5804.04,Nunez,Gerardo


In [238]:
df_template.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_america_national_{date}_.csv', index=False)

# Close Connection DB

In [ ]:
connection.close()